#### Set Up

In [1]:
import pandas as pd
import json
import os
from dotenv import load_dotenv
load_dotenv()

from direct_asking import DirectAsking, DirectAskingAPI
from utils import load_data

c:\Users\ADMIN\.conda\envs\base-12\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
# direct_asking = DirectAsking(
#     model_name='Qwen/Qwen3-4B-Instruct-2507',
#     user_prompt_path='prompts/direct_asking_user.txt',
# )

direct_asking_api = DirectAskingAPI(
    model='gpt-4o-mini-2024-07-18',
    api_key=os.getenv('OPENAI_API_KEY'),
    user_prompt_path='prompts/direct_asking_user.txt',
    retries=3,
)

In [3]:
path = 'data/vihallu-train.csv'
data = load_data(path, 'pandas')
data.head()

,id,context,prompt,response,label
0,9b1ea51d-d1ff-45ba-8cf1-6a91328e8600,"Vào những năm 1870, hai nhà điêu khắc Augustus...","Vào những năm 1960, nơi nào trở thành trung tâ...",Quảng trường Washington là trung tâm của thế h...,extrinsic
1,db7a89c6-2a6a-42af-beef-58e557ecc819,Cách mạng Tháng Mười đã biến một cuộc chiến tr...,Cách mạng Tháng Mười đã khởi xướng chủ nghĩa p...,Sai. Cách mạng Tháng Mười đã đánh bại chủ nghĩ...,no
2,10fca062-d343-4eca-8434-93c7a8aa5e0e,Vị trí địa lý nằm giữa phương Đông và Địa Trun...,Dựa trên việc Đế quốc Ottoman không bao giờ ki...,Nền ẩm thực của Thổ Nhĩ Kỳ được xây dựng từ gi...,extrinsic
3,ece8eb9e-d6bb-407a-a567-d9531861c603,"Hồ Quý Ly trước có tên là Lê Quý Ly (黎季犛), biể...",Đời cháu thứ 12 của Hồ Hưng Dật sống ở đâu?,"Đời cháu thứ 12 của Hồ Hưng Dật, là Hồ Liêm, s...",no
4,b613217f-df2a-491e-8326-25811a31eb09,"Năm 2007, Bồ Đào Nha có khoảng 332.137 người n...","Từ thập niên 1990, tại sao xuất hiện một vài l...",Mặc dù Bồ Đào Nha đã đóng cửa biên giới hoàn t...,intrinsic


#### Benchmark

In [4]:
from tqdm import tqdm

data = data[:1000]
result = []

for id, sample in tqdm(data.iterrows(), total=len(data), desc="Processing samples"):
    label = direct_asking_api.predict(
        context=sample['context'],
        prompt=sample['prompt'],
        response=sample['response']
    )
    result.append({'id': id, 'pred_label': label, 'true_label': sample['label']})

Processing samples: 100%|██████████| 1000/1000 [11:29<00:00,  1.45it/s]


In [5]:
out_path = 'data/output/direct_asking_gpt4o_mini_train.json'

with open(out_path, 'w') as fout:
    json.dump(result, fout)